# Intro to plotly

Probably for a separate tutorial session...

In [1]:
import pandas as pd

In [2]:
from plotly import __version__
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.offline as py
import plotly.graph_objs as go

In [3]:
print(__version__)

3.5.0


In [4]:
py.init_notebook_mode(connected=True)

There are two basic plotting methods:

* `py.plot` generates HTML that is saved locally and viewed in your browser
* `py.iplot` generates HTML that is displayed in a notebook

In [5]:
# When working in a notebook, use iplot rather than plot
py.iplot([go.Scatter(x=[1, 2, 3], y=[3, 1, 6])])

In [7]:
trace0 = go.Scatter(
    x=[1, 2, 3, 4],
    y=[10, 15, 13, 17]
)
trace1 = go.Scatter(
    x=[1, 2, 3, 4],
    y=[16, 5, 11, 9]
)
data = [trace0, trace1]

In [8]:
py.iplot(data)

### Cufflinks allows you to use pandas DataFrames directly with plotly

In [22]:
import cufflinks as cf

In [24]:
df = cf.datagen.lines()
df.head()

,LEZ.LN,ZBM.WU,RKA.AB,TMK.FP,ZFM.LV
2015-01-01,0.920095,-1.041602,-0.392105,0.405764,1.425456
2015-01-02,-1.102762,-0.135624,1.350770,-2.056082,0.665814
2015-01-03,-1.795793,-0.220453,1.395160,-3.294782,1.139757
2015-01-04,-0.349753,-0.046489,3.368866,-2.858586,0.965347
2015-01-05,-0.413891,0.141787,3.688753,-2.082225,0.959571


In [25]:
py.iplot(df.iplot(asFigure=True, kind='scatter', xTitle='Dates', yTitle='Returns', title='Returns'))

### Tableau superstore dataset

In [9]:
try:
    superstore = pd.read_excel('data/superstore.xlsx')
except FileNotFoundError:
    superstore = pd.read_excel('https://query.data.world/s/n2pyux2nabxy4c43zl3uugxsk5gt6v')

In [10]:
superstore.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51290 entries, 0 to 51289
Data columns (total 24 columns):
Row ID            51290 non-null int64
Order ID          51290 non-null object
Order Date        51290 non-null datetime64[ns]
Ship Date         51290 non-null datetime64[ns]
Ship Mode         51290 non-null object
Customer ID       51290 non-null object
Customer Name     51290 non-null object
Segment           51290 non-null object
Postal Code       9994 non-null float64
City              51290 non-null object
State             51290 non-null object
Country           51290 non-null object
Region            51290 non-null object
Market            51290 non-null object
Product ID        51290 non-null object
Category          51290 non-null object
Sub-Category      51290 non-null object
Product Name      51290 non-null object
Sales             51290 non-null float64
Quantity          51290 non-null int64
Discount          51290 non-null float64
Profit            51290 non-null fl

In [11]:
superstore.Segment.value_counts()

Consumer       26518
Corporate      15429
Home Office     9343
Name: Segment, dtype: int64

### Plot a line chart of sales by segment

#### First, just do one series

In [12]:
# Summarise by month, then group by Segment so we can iterate through the groups
sales = superstore.set_index('Order Date').groupby('Segment').resample('1M')['Sales'].sum().reset_index().groupby('Segment')

In [13]:
# Get just the Consumer segment
group = sales.get_group('Consumer') 

In [14]:
data = [go.Scatter(x=group['Order Date'], y=group['Sales'], name='Consumer')]

In [16]:
py.iplot(data)

#### Now several series

In [18]:
data = [go.Scatter(x=group['Order Date'], y=group['Sales'], name=key) for key, group in sales]

In [19]:
# Set the layout
layout = dict(title = 'Monthly sales by segment',
              xaxis = dict(title = 'Date'),
              yaxis = dict(title = 'Sales ($)'),
              )

In [21]:
fig = dict(data=data, layout=layout)
py.iplot(fig)

### Now make the chart interactive - display one line at a time

Based on the plotly examples [here](https://plot.ly/python/dropdowns/)

In [44]:
superstore.Segment.value_counts()

Consumer       26518
Corporate      15429
Home Office     9343
Name: Segment, dtype: int64

In [142]:
updatemenus = list([
    dict(active=3, # Sets the initially active item
         type='buttons',
         direction='right',
         xanchor='left',
         yanchor='top',
         buttons=list([   
            dict(label = 'Consumer',
                 method = 'update',
                 args = [{'visible': [True, False, False], 'title': 'Consumer'}]),
            dict(label = 'Corporate',
                 method = 'update',
                 args = [{'visible': [False, True, False], 'title': 'Corporate'}]),
            dict(label = 'Home Office',
                 method = 'update',
                 args = [{'visible': [False, False, True], 'title': 'Home Office'}]),
            dict(label = 'All',
                 method = 'update',
                 args = [{'visible': [True, True, True], 'title': 'All Segments'}]),
        ]),
    )
])


In [143]:
updatemenus

[{'active': 3,
  'type': 'buttons',
  'direction': 'right',
  'xanchor': 'left',
  'yanchor': 'top',
  'buttons': [{'label': 'Consumer',
    'method': 'update',
    'args': [{'visible': [True, False, False], 'title': 'Consumer'}]},
   {'label': 'Corporate',
    'method': 'update',
    'args': [{'visible': [False, True, False], 'title': 'Corporate'}]},
   {'label': 'Home Office',
    'method': 'update',
    'args': [{'visible': [False, False, True], 'title': 'Home Office'}]},
   {'label': 'All',
    'method': 'update',
    'args': [{'visible': [True, True, True], 'title': 'All Segments'}]}]}]

In [144]:
# Better version
updatemenus = list([
    dict(active=3, 
         type='buttons',
         direction='right',
         xanchor='left',
         yanchor='top')
])

In [152]:
# Now add buttons
b = []
segments = superstore.Segment.unique()

for s in segments:
    d = dict(label = s,
             method = 'update', 
             args = [{'visible': [s==segments], 'title': s}]),
    b.append(d)

In [153]:
b

[({'label': 'Consumer',
   'method': 'update',
   'args': [{'visible': [array([ True, False, False])],
     'title': 'Consumer'}]},),
 ({'label': 'Corporate',
   'method': 'update',
   'args': [{'visible': [array([False,  True, False])],
     'title': 'Corporate'}]},),
 ({'label': 'Home Office',
   'method': 'update',
   'args': [{'visible': [array([False, False,  True])],
     'title': 'Home Office'}]},)]

In [151]:
b[0]

({'label': 'Consumer',
  'method': 'update',
  'args': [{'visible': [array([ True, False, False])]},
   {'title': 'Consumer'}]},)

In [146]:
updatemenus[0]['buttons'] = b

In [147]:
updatemenus 

[{'active': 3,
  'type': 'buttons',
  'direction': 'right',
  'xanchor': 'left',
  'yanchor': 'top',
  'buttons': [({'label': 'Consumer',
     'method': 'update',
     'args': [{'visible': [array([ True, False, False])]},
      {'title': 'Consumer'}]},),
   ({'label': 'Corporate',
     'method': 'update',
     'args': [{'visible': [array([False,  True, False])]},
      {'title': 'Corporate'}]},),
   ({'label': 'Home Office',
     'method': 'update',
     'args': [{'visible': [array([False, False,  True])]},
      {'title': 'Home Office'}]},)]}]

In [140]:
# Set the layout
layout = dict(title = 'Monthly sales by segment',
              xaxis = dict(title = 'Date'),
              yaxis = dict(title = 'Sales ($)'),
              )
layout['updatemenus'] = updatemenus

In [141]:
fig = dict(data=data, layout=layout)
py.iplot(fig)

ValueError: 
    Invalid element(s) received for the 'buttons' property of layout.updatemenu
        Invalid elements include: [({'label': 'Consumer', 'method': 'update', 'args': [{'visible': [array([ True, False, False])]}, {'title': 'Consumer'}]},), ({'label': 'Corporate', 'method': 'update', 'args': [{'visible': [array([False,  True, False])]}, {'title': 'Corporate'}]},), ({'label': 'Home Office', 'method': 'update', 'args': [{'visible': [array([False, False,  True])]}, {'title': 'Home Office'}]},)]

    The 'buttons' property is a tuple of instances of
    Button that may be specified as:
      - A list or tuple of instances of plotly.graph_objs.layout.updatemenu.Button
      - A list or tuple of dicts of string/value properties that
        will be passed to the Button constructor

        Supported dict properties:
            
            args
                Sets the arguments values to be passed to the
                Plotly method set in `method` on click.
            execute

SyntaxError: invalid syntax (<ipython-input-113-f6460d7d9d2b>, line 1)